In [16]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [17]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings


In [18]:
# Load your document
loader = TextLoader("docs/environment.txt")  # Replace with your filename
documents = loader.load()

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

# Create embeddings using Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Vector store using FAISS
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_index")


In [22]:
# Load vectorstore again
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


# Gemini chat model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.1)

# Memory to store conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create Conversational RAG Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory
)


In [ ]:
# Simple terminal-like chat inside notebook
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    result = qa_chain({"question": query})
    print("Bot:", result["answer"])


You:  hi


Bot: Hi there! How can I help you today?


You:  tell me about the weather of lahore


Bot: Lahore is experiencing the effects of climate change, including an increased frequency of heatwaves with temperatures occasionally exceeding 45°C (113°F). The city also suffers from the "urban heat island" effect, making it warmer than surrounding rural areas.


You:  famous places in lahore


Bot: I'm sorry, but the context provided does not contain information about famous places in Lahore. It mainly focuses on the environmental challenges and causes of air pollution in the city.


You:  what are major reasons for air pollution in lahore


Bot: Based on the provided document, the major reasons for air pollution in Lahore are:

*   **Vehicular Emissions**: A large number of vehicles, often using poor-quality fuel and lacking emission controls.
*   **Industrial Activity**: Emissions from unregulated industries like brick kilns and steel rerolling mills.
*   **Crop Burning**: Smoke and particulate matter from agricultural burning in nearby areas.
